In [ ]:
import os
import cv2
import json
import numpy as np
import urllib.request

def load_json_from_scalabel():
    scalabel_data = []
    scalabel_path = '2019-01_vehicle_damages_Results.json'
    for json_file in sorted(os.listdir(scalabel_path)):
        with open(os.path.join(scalabel_path, json_file)) as f:
            data = json.load(f)
            scalabel_data.extend(data)
    return scalabel_data
load_json_from_scalabel()

In [ ]:
def url_to_image(url):
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    return image
url_to_image()

In [ ]:
def download_images_from_s3():
    scalabel_data = load_json_from_scalabel()
    for index, img in enumerate(scalabel_data):
        image = url_to_image(img['name'])
        image_name = img['name'].replace('/', '_')
        cv2.imwrite(image_name, image)
    print(index, len(scalabel_data))
download_images_from_s3()

In [ ]:
def extract_params_from_scalabel():
    points = []
    scalabel_data = load_json_from_scalabel()
    img_path = ''
    label = []
    height = []
    width = []
    for i, img in enumerate(scalabel_data):
        try:
            for anno in img['labels']:
                poly2d_data = anno['poly2d']
                label.append(anno['category'])
                for p in poly2d_data:
                    points.append(p['vertices'])
                image_name = img['name'].replace('/', '_')
                img = cv2.imread(img_path + image_name)
                h, w = img.shape[:2] # This is imageHeight, imageWidth
                height.append(h)
                width.append(w)
        except:
            pass
    return label,height, width, points
extract_params_from_scalabel()